# * Adhoc : Target & Baseline %EBITDA

## Parameter

In [2]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb


config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2024, 6, 17)

## ETL Process

### Step 1 : Import Data Source (Excel)
    Read Excel data

In [4]:
# Source Config
src_file = 'C:\Ruz\Pentaho\Jobs\Input\Target\ADHOC-RAW_EBITDA_TARGET & BASELINE.xlsx'
src_sheet = 'Target & Baseline'
src_skiprows = 2
src_usecols = 'B:L'

# Create DataFrame
src_df = pd.read_excel(src_file, sheet_name=src_sheet, skiprows=src_skiprows, usecols=src_usecols, index_col=None) 
src_df = src_df.replace(np.nan, None)
# src_df.tail(3)

In [5]:
# src_df = src_df[src_df['USE_FLAG']=='Y']
# src_df[['METRIC_CD', 'METRIC_NAME']].drop_duplicates().sort_values('METRIC_CD').reset_index(drop=True)
# src_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg(START_MTH=('TM_KEY_MTH', np.min), END_MTH=('TM_KEY_MTH', np.max))
x = src_df.groupby(['COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'TM_KEY_MTH': ['min','max'], 'AREA_TYPE': 'nunique'})
x

TM_KEY_MTH         AREA_TYPE
                                     min     max   nunique
COMP_CD METRIC_CD METRIC_NAME                             
ALL     CAP00006  %EBITDA         202401  202404         1

In [6]:
src_df.tail(3)

,TM_KEY_MTH,COMP_CD,METRIC_CD,METRIC_NAME,METRIC_VALUE,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,REMARK
517,202404,ALL,CAP00006,%EBITDA,32.500,B,3,H,96X,"PATTANI, YALA, NARATHIWAT",None
518,202404,ALL,CAP00006,%EBITDA,33.725,B,3,H,23Y,"CHANTHABURI, TRAT",None
519,202404,ALL,CAP00006,%EBITDA,37.567,B,3,H,003,"SMP : Bang Bo, Bang Sao Thong, Bang Phli",None


### Step 2 : Import dim_time.csv
    Read CSV data

In [7]:
# Create DataFrame

# src_df = pd.read_excel(src_file, sheet_name=src_sheet, skiprows=src_skiprows, usecols=src_usecols, index_col=None) 

dt_file = '../CFW/data/dim_time.csv'
dt_df = pd.read_csv(dt_file)

dt_df.tail(3)

,TM_KEY_DAY,DAY_NO,DATE_VALUE,DAY_SHORT,DAY_OF_WEEK,DAYS_IN_MONTH,TRUE_TM_KEY_WK,TRUE_WEEK,TM_KEY_WK,WEEK_YEAR,TM_KEY_MTH,MONTH_NO,TM_KEY_QTR,QUARTER_NO,TM_KEY_YR,PERIODFLAG,LOAD_DATE
1093,20251229,29,2025-12-29,Mon,1,31,2026001,1,2025053,53,202512,12,20254,4,2025,N,2024-06-17 00:01:19
1094,20251230,30,2025-12-30,Tue,2,31,2026001,1,2025053,53,202512,12,20254,4,2025,N,2024-06-17 00:01:19
1095,20251231,31,2025-12-31,Wed,3,31,2026001,1,2025053,53,202512,12,20254,4,2025,EMQY,2024-06-17 00:01:19
